In [ ]:
from common import utils

DBX_CTX = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
API_URL = DBX_CTX.apiUrl().getOrElse(None)
API_TOKEN = DBX_CTX.apiToken().getOrElse(None)

CATALOG_NAME = utils.config_value("catalog_name")
SCHEMA_NAME = utils.config_value("schema_name")
KNOWLEDGE_ASSISTANT_NAME = utils.snake_case(f"{CATALOG_NAME}_{SCHEMA_NAME}_knowledge_assistant")

In [ ]:
import json
import requests
from common import vector_search_config

url = f"{API_URL}/api/2.0/knowledge-assistants"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_TOKEN}",
}

payload = {
    "name": KNOWLEDGE_ASSISTANT_NAME,
    "description": utils.config_value("knowledge_assistant_description"),
    "instructions": utils.config_value("knowledge_assistant_instructions", ""),
    "knowledge_sources": [
        {
            "files_source": {
                "name": "files",
                "type": "files",
                "description": utils.config_value("file_source_description"),
                "files": {
                    "path": vector_search_config.VOLUME_PATH
                }
            }
        },
        {
            "index_source": {
                "name": "vector_search_index",
                "type": "index",
                "description": utils.config_value("vector_search_index_description"),
                "index": {
                    "name": vector_search_config.INDEX_NAME,
                    "doc_uri_col": "path",
                    "text_col": "text"
                }
            }
        }
    ]
}

print(json.dumps(payload, indent=2))

response = requests.post(url, headers=headers, json=payload)
print(response.text)
response.raise_for_status()

print(response.status_code)

